In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-02.csv')

C:\Users\Bro Bro\.conda\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 819.8717083930969 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 833.6771290302277 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,VTS,2014-02-09 14:51:00,2014-02-09 14:58:00,1,1.06,-73.982370,40.768287,1,NaN,-73.977575,40.758175,CSH,6.5,0.0,0.5,0.0,0.0,7.0,POINT (-73.977575 40.758175),POINT (-73.98237 40.768287)
1,VTS,2014-02-06 14:27:00,2014-02-06 14:44:00,2,1.48,-73.983070,40.764430,1,NaN,-73.993750,40.750330,CRD,11.0,0.0,0.5,1.0,0.0,12.5,POINT (-73.99375000000001 40.75033),POINT (-73.98307 40.76443)
2,VTS,2014-02-06 14:22:00,2014-02-06 14:45:00,1,2.84,-74.002667,40.723770,1,NaN,-73.982272,40.757965,CRD,15.5,0.0,0.5,3.1,0.0,19.1,POINT (-73.98227199999998 40.757965),POINT (-74.002667 40.72377)


In [10]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [11]:
df2_end=df[['dropoff_geometry']]

In [12]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 113.441153049469 seconds ---


In [13]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 112.69771599769592 seconds ---


In [ ]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
geo_processing_start

In [ ]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [ ]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [ ]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [ ]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [ ]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [ ]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-02-processed.csv')
complete

In [ ]:
print("--- %s seconds ---" % (time.time() - full_time))